In [1]:
import numpy as np # đại số tuyến tính
import pandas as pd # xử lý dữ liệu, I/O tệp CSV (ví dụ: pd.read_csv)
import matplotlib.pyplot as plt # cho mục đích trực quan hóa dữ liệu
import seaborn as sns # để trực quan hóa dữ liệu thống kê
%matplotlib inline
df = pd.read_excel("data\\dataNCKH.xlsx")
df.head()

,symptom_1,symptom_2,symptom_3,symptom_4,diagnose
0,đau họng,khó nuốt,ho kéo dài,khạc dàm dai dẳng,viêm họng
1,đau bụng,tiêu chảy,buồn nôn,đau đầu,viêm dạ dày
2,đau lưng,đau cơ,đau đầu,buồn nôn,căng thẳng cơ
3,sổ mũi,đau mũi,ho,mệt mỏi,cảm lạnh
4,đau rụt bụng dưới,tiểu tiện đau,tiểu tiện buốt,tiểu tiện nhiều,viêm bàng quang


In [2]:
col_names = ['symptom_1', 'symptom_2', 'symptom_3', 'symptom_4','diagnose']

df.columns = col_names

df.columns

Index(['symptom_1', 'symptom_2', 'symptom_3', 'symptom_4', 'diagnose'], dtype='object')

In [3]:
df.isnull().sum()

symptom_1    0
symptom_2    0
symptom_3    0
symptom_4    0
diagnose     0
dtype: int64

In [4]:
categorical = [var for var in df.columns if df[var].dtype=='O']

print('Có {} biến phân loại\n'.format(len(categorical)))

print('Các biến phân loại là :\n\n', categorical)

Có 5 biến phân loại

Các biến phân loại là :

 ['symptom_1', 'symptom_2', 'symptom_3', 'symptom_4', 'diagnose']


In [5]:
df[categorical].isnull().sum()

symptom_1    0
symptom_2    0
symptom_3    0
symptom_4    0
diagnose     0
dtype: int64

In [6]:
X = df.drop(['diagnose'], axis=1)
y = df['diagnose']

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [8]:
!pip install category_encoders
import category_encoders as ce


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Xác định các biến phân loại
categorical = ['symptom_1', 'symptom_2', 'symptom_3', 'symptom_4']

# Tạo encoder
encoder = ce.OneHotEncoder(cols=categorical)

# Thực hiện one-hot encoding trên tập huấn luyện
X_train = encoder.fit_transform(X_train)

# Áp dụng biến đổi đã học từ tập huấn luyện lên tập kiểm tra
X_test = encoder.transform(X_test)

In [10]:
from sklearn.naive_bayes import GaussianNB

# instantiate the model
gnb = GaussianNB()
# fit the model
gnb.fit(X_train, y_train)

GaussianNB()

In [11]:
y_pred = gnb.predict(X_test)

y_pred

array(['cảm lạnh', ' covid 19', 'nhiễm khuẩn dạ dày', ' covid 19',
       ' covid 19', 'đau đầu thống kinh', ' covid 19', ' sốt xuất huyết',
       ' sốt xuất huyết', 'đau tim', ' sốt xuất huyết',
       'đau đầu căng thẳng', ' covid 19', ' covid 19', 'viêm phổi',
       ' covid 19', ' covid 19', ' sốt xuất huyết', ' covid 19',
       'căng thẳng cơ', 'huyết áp cao', ' covid 19', 'đau đầu thống kinh',
       ' covid 19', 'viêm khớp', ' covid 19', ' sốt xuất huyết',
       ' sốt xuất huyết', 'viêm mũi họng', 'căng thẳng', 'viêm dạ dày',
       'viêm dạ dày', 'sốt xuất huyết', ' covid 19', 'viêm amidan',
       'căng thẳng', ' covid 19', 'đau đầu thống kinh', ' sốt xuất huyết',
       ' covid 19', 'đau nửa đầu', ' covid 19', ' sốt xuất huyết',
       'viêm mắt', 'đau đầu căng thẳng', ' covid 19', ' covid 19',
       'viêm khớp', ' covid 19', 'cảm lạnh', ' covid 19', 'căng thẳng cơ',
       'viêm tai giữa', ' covid 19', ' covid 19', 'cảm lạnh',
       'đau đầu căng thẳng', 'viêm dạ dày', 

In [12]:
from sklearn.metrics import accuracy_score

print('Độ chính xác của mô hình: {0:0.4f} %'. format(accuracy_score(y_test, y_pred)*100))

Độ chính xác của mô hình: 96.9136 %


In [13]:
print('Điểm training: {:.4f} %'.format(gnb.score(X_train, y_train)*100))

print('Điểm test: {:.4f} %'.format(gnb.score(X_test, y_test)*100))

Điểm training: 97.8667 %
Điểm test: 96.9136 %


In [14]:
# # Thêm dòng mới vào DataFrame
new_data = {
    'symptom_1':['nhức mắt'], 
    'symptom_2':['đau đầu'], 
    'symptom_3':['sưng mí mắt'], 
    'symptom_4':['say nắng'],
}
new_df = pd.DataFrame(new_data)

# Áp dụng encoder đã học từ tập huấn luyện
new_df_encoded = encoder.transform(new_df)

# Dự đoán bằng mô hình đã huấn luyện
new_prediction = gnb.predict(new_df_encoded)

# In kết quả dự đoán
print("Kết quả dự đoán: ", new_prediction)

Kết quả dự đoán:  ['cảm lạnh']


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_dt_pred = dt.predict(X_test)

dt_accuracy = accuracy_score(y_test, y_dt_pred)
print('Độ chính xác của Decision Tree: {:.4f} %'.format(dt_accuracy * 100))

dt_classification_report = classification_report(y_test, y_dt_pred)
print('Báo cáo phân loại của Decision Tree:\n', dt_classification_report)

dt_confusion_matrix = confusion_matrix(y_test, y_dt_pred)
print('Ma trận nhầm lẫn của Decision Tree:\n', dt_confusion_matrix)


Độ chính xác của Decision Tree: 96.2963 %
Báo cáo phân loại của Decision Tree:
                     precision    recall  f1-score   support

          covid 19       1.00      0.93      0.97        61
    căng thẳng cơ        0.00      0.00      0.00         0
    sốt xuất huyết       0.87      1.00      0.93        33
        căng thẳng       1.00      1.00      1.00         2
     căng thẳng cơ       1.00      0.75      0.86         4
          cảm lạnh       1.00      1.00      1.00         6
            dị ứng       1.00      1.00      1.00         1
      huyết áp cao       1.00      1.00      1.00         1
nhiễm khuẩn dạ dày       1.00      1.00      1.00         5
          say nắng       1.00      1.00      1.00         1
          suy thận       0.00      0.00      0.00         1
    sốt xuất huyết       1.00      1.00      1.00         1
       viêm amidan       1.00      1.00      1.00         1
   viêm bàng quang       1.00      1.00      1.00         1
       viêm dạ dày 

c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [16]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)
y_svm_pred = svm.predict(X_test)

svm_accuracy = accuracy_score(y_test, y_svm_pred)
print('Độ chính xác của SVM: {:.4f} %'.format(svm_accuracy * 100))

svm_classification_report = classification_report(y_test, y_svm_pred)
print('Báo cáo phân loại của SVM:\n', svm_classification_report)

svm_confusion_matrix = confusion_matrix(y_test, y_svm_pred)
print('Ma trận nhầm lẫn của SVM:\n', svm_confusion_matrix)


Độ chính xác của SVM: 97.5309 %
Báo cáo phân loại của SVM:
                     precision    recall  f1-score   support

          covid 19       1.00      1.00      1.00        61
    căng thẳng cơ        0.00      0.00      0.00         0
    sốt xuất huyết       0.92      1.00      0.96        33
        căng thẳng       1.00      1.00      1.00         2
     căng thẳng cơ       1.00      0.75      0.86         4
          cảm lạnh       1.00      1.00      1.00         6
            dị ứng       1.00      1.00      1.00         1
      huyết áp cao       1.00      1.00      1.00         1
nhiễm khuẩn dạ dày       1.00      1.00      1.00         5
          say nắng       1.00      1.00      1.00         1
          suy thận       0.00      0.00      0.00         1
    sốt xuất huyết       1.00      1.00      1.00         1
       viêm amidan       1.00      1.00      1.00         1
   viêm bàng quang       1.00      1.00      1.00         1
       viêm dạ dày       1.00      1.00

c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf_pred = rf.predict(X_test)

rf_accuracy = accuracy_score(y_test, y_rf_pred)
print('Độ chính xác của Random Forest: {:.4f} %'.format(rf_accuracy * 100))

rf_classification_report = classification_report(y_test, y_rf_pred)
print('Báo cáo phân loại của Random Forest:\n', rf_classification_report)

rf_confusion_matrix = confusion_matrix(y_test, y_rf_pred)
print('Ma trận nhầm lẫn của Random Forest:\n', rf_confusion_matrix)


Độ chính xác của Random Forest: 98.1481 %
Báo cáo phân loại của Random Forest:
                     precision    recall  f1-score   support

          covid 19       0.97      1.00      0.98        61
    căng thẳng cơ        0.00      0.00      0.00         0
    sốt xuất huyết       1.00      0.97      0.98        33
        căng thẳng       1.00      1.00      1.00         2
     căng thẳng cơ       1.00      0.75      0.86         4
          cảm lạnh       1.00      1.00      1.00         6
            dị ứng       1.00      1.00      1.00         1
      huyết áp cao       1.00      1.00      1.00         1
nhiễm khuẩn dạ dày       1.00      1.00      1.00         5
          say nắng       1.00      1.00      1.00         1
          suy thận       0.00      0.00      0.00         1
    sốt xuất huyết       1.00      1.00      1.00         1
       viêm amidan       1.00      1.00      1.00         1
   viêm bàng quang       1.00      1.00      1.00         1
       viêm dạ dày 

c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\ducva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie